In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm_notebook
from tqdm import tqdm
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

### 크롤링

##### 구 번호 (seoul_gu)

In [2]:
driver = webdriver.Chrome()
driver.get('https://www.weather.go.kr/w/obs-climate/land/aws-obs.do?db=DAYDB&tm=2024.03.01%2000%3A00&stnId=400&sidoCode=asos&sort=&config=')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR,'#aws-sido').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR,'#aws-sido > option:nth-child(9)').click()
time.sleep(2)
driver.find_element(By.CSS_SELECTOR,'#aws-panel-wrap > div.aws-panel-head > div:nth-child(4) > a').click()
req=driver.page_source
soup = BeautifulSoup(req,'html.parser')
seoul_gu = soup.select('select[name="sido-stn"] option') #html(soup)에서 전체만 추출
seoul_gu = [option['value'] for option in seoul_gu]
seoul_gu

['400',
 '402',
 '424',
 '404',
 '509',
 '413',
 '423',
 '417',
 '410',
 '110',
 '425',
 '407',
 '406',
 '408',
 '411',
 '412',
 '108',
 '401',
 '421',
 '414',
 '403',
 '405',
 '510',
 '415',
 '416',
 '419',
 '409',
 '418',
 '889']

##### 구 이름(seoul_gu_name)

In [4]:
select_element = soup.find('select', id='aws-sido-stn')

# 옵션 값과 텍스트를 저장할 리스트 생성
seoul_gu_name = []

# 옵션 값과 텍스트 추출하여 리스트에 추가
options = select_element.find_all('option')
for option in options:
    value = option['value']
    text = option.get_text(strip=True)
    seoul_gu_name.append((value, text))

driver.quit()
seoul_gu_name

[('400', '강남'),
 ('402', '강동'),
 ('424', '강북*'),
 ('404', '강서'),
 ('509', '관악'),
 ('413', '광진'),
 ('423', '구로'),
 ('417', '금천'),
 ('410', '기상청'),
 ('110', '김포(공)'),
 ('425', '남현'),
 ('407', '노원'),
 ('406', '도봉'),
 ('408', '동대문'),
 ('411', '마포'),
 ('412', '서대문'),
 ('108', '서울'),
 ('401', '서초'),
 ('421', '성동'),
 ('414', '성북'),
 ('403', '송파 *'),
 ('405', '양천'),
 ('510', '영등포 *'),
 ('415', '용산'),
 ('416', '은평'),
 ('419', '중구'),
 ('409', '중랑'),
 ('418', '한강'),
 ('889', '현충원')]

##### 날씨 정보 추출(df_2020~2023)

In [5]:
driver = webdriver.Chrome()
years = ['2023','2022','2021','2020']
months = ['02','03','04'] #'일'은 1일로 고정, 따라서 04월 01일 ~03월까지, 03월 01일 ~ 02월까지
data = []
for year in years:
    for month in months:
        for gu in seoul_gu:
            # 해당 웹페이지로 이동
            url = f'https://www.weather.go.kr/w/obs-climate/land/aws-obs.do?db=DAYDB&tm={year}.{month}.01%2000%3A00&stnId={gu}&sidoCode=asos&sort=&config='
            driver.get(url)
            
            # 페이지 소스를 BeautifulSoup로 파싱
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # 표의 데이터 추출

            for row in soup.find_all('tr'):
                cols = row.find_all(['td', 'th'])
                cols = [col.text.strip() for col in cols]
                data.append(cols + [gu])
            
            if data:
                df = pd.DataFrame(data[1:], columns=data[0])
                print(df)
            else:
                print(f"No data found for {year}-{month} and gu {gu}")

# WebDriver 종료
driver.quit()

           년월일 최저기온(℃) 최고기온(℃) 일강수량(mm) 최대순간풍속(m/s)  400
0   2023-02-01    -1.7     6.9      0.0         8.7  400
1   2023-01-31    -3.8     9.4      0.0         7.0  400
2   2023-01-30    -3.5     4.6      0.0         7.8  400
3   2023-01-29    -8.1     5.8      0.0         7.5  400
4   2023-01-28   -10.0    -0.1      0.0         6.9  400
5   2023-01-27    -7.9    -2.7      0.0         9.4  400
6   2023-01-26    -8.9    -3.1      0.0         4.6  400
7   2023-01-25   -16.2    -5.5      0.0         5.4  400
8   2023-01-24   -15.9    -5.5      0.0         9.7  400
9   2023-01-23    -5.5     5.8      0.0         6.9  400
10  2023-01-22    -2.4     3.5      0.0         8.5  400
11  2023-01-21    -8.6     1.7      0.0         4.7  400
12  2023-01-20    -5.5     3.3      0.0        10.1  400
13  2023-01-19    -4.9     5.3      2.0         6.9  400
14  2023-01-18    -3.3     3.1      0.0         6.2  400
15  2023-01-17    -5.5     0.5      0.0         4.1  400
16  2023-01-16    -5.1    -0.2 

In [6]:
df

,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s),400
0,2023-02-01,-1.7,6.9,0.0,8.7,400
1,2023-01-31,-3.8,9.4,0.0,7.0,400
2,2023-01-30,-3.5,4.6,0.0,7.8,400
3,2023-01-29,-8.1,5.8,0.0,7.5,400
4,2023-01-28,-10.0,-0.1,0.0,6.9,400
...,...,...,...,...,...,...
10714,2020-03-06,-4.5,11.3,0.0,4.5,889
10715,2020-03-05,-1.9,7.6,0.0,6.7,889
10716,2020-03-04,-0.6,6.1,0.0,9.4,889
10717,2020-03-03,-2.0,12.5,0.0,7.0,889


##### 날씨 정보 추출(df24_2024,오늘 날짜 3.20까지)

In [7]:
driver = webdriver.Chrome()
months = ['02','03']
data24 = []
for year in years:
    for month in months:
        for gu in seoul_gu:
            # 해당 웹페이지로 이동
            url = f'https://www.weather.go.kr/w/obs-climate/land/aws-obs.do?db=DAYDB&tm=2024.{month}.20%2000%3A00&stnId={gu}&sidoCode=asos&sort=&config='
            driver.get(url)
            
            # 페이지 소스를 BeautifulSoup로 파싱
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # 표의 데이터 추출

            for row in soup.find_all('tr'):
                cols = row.find_all(['td', 'th'])
                cols = [col.text.strip() for col in cols]
                data24.append(cols + [gu])
            
            if data:
                df24 = pd.DataFrame(data24[1:], columns=data[0])
                print(df24)
            else:
                print(f"No data found for 2024-{month} and gu {gu}")

# WebDriver 종료
driver.quit()

           년월일 최저기온(℃) 최고기온(℃) 일강수량(mm) 최대순간풍속(m/s)  400
0   2024-02-20     4.2     8.3      2.5         5.3  400
1   2024-02-19     5.3    12.4      7.0         7.6  400
2   2024-02-18     0.0    10.5     16.0         9.2  400
3   2024-02-17    -1.0    12.0      0.0         3.9  400
4   2024-02-16    -1.4     8.3      0.0         5.3  400
5   2024-02-15     1.8    12.1      9.0         8.5  400
6   2024-02-14     8.1    18.9      0.0         6.1  400
7   2024-02-13     0.7    15.2      0.0         8.0  400
8   2024-02-12    -2.7    11.2      0.0         5.1  400
9   2024-02-11    -3.0     6.7      0.0         8.8  400
10  2024-02-10    -2.6     7.5      0.0         8.3  400
11  2024-02-09    -2.8     6.6      0.0         7.7  400
12  2024-02-08    -3.4     6.6      0.0         5.8  400
13  2024-02-07    -1.2     3.5      0.0         4.5  400
14  2024-02-06     0.2     4.3      0.0         5.8  400
15  2024-02-05     0.3     7.1      4.0         7.4  400
16  2024-02-04     3.6    12.0 

In [8]:
df24

,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s),400
0,2024-02-20,4.2,8.3,2.5,5.3,400
1,2024-02-19,5.3,12.4,7.0,7.6,400
2,2024-02-18,0.0,10.5,16.0,9.2,400
3,2024-02-17,-1.0,12.0,0.0,3.9,400
4,2024-02-16,-1.4,8.3,0.0,5.3,400
...,...,...,...,...,...,...
7374,2024-02-23,-0.7,6.6,0.0,5.2,889
7375,2024-02-22,-0.6,2.2,10.5,6.3,889
7376,2024-02-21,0.7,4.9,19.0,5.9,889
7377,2024-02-20,3.4,8.5,2.5,5.9,889


### 전처리

In [9]:
def pre(dataframe):
    df_pre=dataframe.copy()
    df_pre=df_pre.rename(columns={'400' : '지역 코드'})
    for n in range(len(seoul_gu_name)):
        gu_code = seoul_gu_name[n][0]
        gu_name = seoul_gu_name[n][1]
        df_pre.loc[df_pre['지역 코드'] == gu_code, '지역명'] = gu_name
    # '년월일' 형변환 (from 'object' to 'datetime')
    df_pre['년월일'] = pd.to_datetime(df_pre['년월일'], errors='coerce', format='%Y-%m-%d')

    #'년월일' 공백이면 행 삭제
    df_pre = df_pre.dropna(subset=['년월일'])

    # 중복행 제거
    df_pre.drop_duplicates(subset=['지역 코드', '년월일'], keep='first', inplace=True)
    return df_pre

In [10]:
def col_order(dataframe):
    new_order_column=['지역명','지역 코드','년월일', '최저기온(℃)', '최고기온(℃)', '일강수량(mm)', '최대순간풍속(m/s)']
    dataframe=dataframe[new_order_column]
    return dataframe

In [12]:
df_pre=pre(df)
df_pre

,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s),지역 코드,지역명
0,2023-02-01,-1.7,6.9,0.0,8.7,400,강남
1,2023-01-31,-3.8,9.4,0.0,7.0,400,강남
2,2023-01-30,-3.5,4.6,0.0,7.8,400,강남
3,2023-01-29,-8.1,5.8,0.0,7.5,400,강남
4,2023-01-28,-10.0,-0.1,0.0,6.9,400,강남
...,...,...,...,...,...,...,...
10714,2020-03-06,-4.5,11.3,0.0,4.5,889,현충원
10715,2020-03-05,-1.9,7.6,0.0,6.7,889,현충원
10716,2020-03-04,-0.6,6.1,0.0,9.4,889,현충원
10717,2020-03-03,-2.0,12.5,0.0,7.0,889,현충원


In [13]:
df24_pre=pre(df24)
df24_pre

,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s),지역 코드,지역명
0,2024-02-20,4.2,8.3,2.5,5.3,400,강남
1,2024-02-19,5.3,12.4,7.0,7.6,400,강남
2,2024-02-18,0.0,10.5,16.0,9.2,400,강남
3,2024-02-17,-1.0,12.0,0.0,3.9,400,강남
4,2024-02-16,-1.4,8.3,0.0,5.3,400,강남
...,...,...,...,...,...,...,...
2932,2024-02-25,1.1,6.5,0.0,5.0,413,광진
2933,2024-02-24,-0.7,6.1,0.0,6.4,413,광진
2934,2024-02-23,-0.6,6.2,0.0,5.8,413,광진
2935,2024-02-22,-0.8,1.9,3.0,6.2,413,광진


In [14]:
df_pre=col_order(df_pre)
df_pre

,지역명,지역 코드,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s)
0,강남,400,2023-02-01,-1.7,6.9,0.0,8.7
1,강남,400,2023-01-31,-3.8,9.4,0.0,7.0
2,강남,400,2023-01-30,-3.5,4.6,0.0,7.8
3,강남,400,2023-01-29,-8.1,5.8,0.0,7.5
4,강남,400,2023-01-28,-10.0,-0.1,0.0,6.9
...,...,...,...,...,...,...,...
10714,현충원,889,2020-03-06,-4.5,11.3,0.0,4.5
10715,현충원,889,2020-03-05,-1.9,7.6,0.0,6.7
10716,현충원,889,2020-03-04,-0.6,6.1,0.0,9.4
10717,현충원,889,2020-03-03,-2.0,12.5,0.0,7.0


In [15]:
df24_pre=col_order(df24_pre)
df24_pre

,지역명,지역 코드,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s)
0,강남,400,2024-02-20,4.2,8.3,2.5,5.3
1,강남,400,2024-02-19,5.3,12.4,7.0,7.6
2,강남,400,2024-02-18,0.0,10.5,16.0,9.2
3,강남,400,2024-02-17,-1.0,12.0,0.0,3.9
4,강남,400,2024-02-16,-1.4,8.3,0.0,5.3
...,...,...,...,...,...,...,...
2932,광진,413,2024-02-25,1.1,6.5,0.0,5.0
2933,광진,413,2024-02-24,-0.7,6.1,0.0,6.4
2934,광진,413,2024-02-23,-0.6,6.2,0.0,5.8
2935,광진,413,2024-02-22,-0.8,1.9,3.0,6.2


### CONCAT

In [16]:
df_combined = pd.concat([df_pre, df24_pre], ignore_index=True)
df_combined

,지역명,지역 코드,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s)
0,강남,400,2023-02-01,-1.7,6.9,0.0,8.7
1,강남,400,2023-01-31,-3.8,9.4,0.0,7.0
2,강남,400,2023-01-30,-3.5,4.6,0.0,7.8
3,강남,400,2023-01-29,-8.1,5.8,0.0,7.5
4,강남,400,2023-01-28,-10.0,-0.1,0.0,6.9
...,...,...,...,...,...,...,...
11821,광진,413,2024-02-25,1.1,6.5,0.0,5.0
11822,광진,413,2024-02-24,-0.7,6.1,0.0,6.4
11823,광진,413,2024-02-23,-0.6,6.2,0.0,5.8
11824,광진,413,2024-02-22,-0.8,1.9,3.0,6.2


### sort_value

In [17]:
df_combined=df_combined.sort_values(by=['지역 코드', '년월일'], ascending=[True, True])
df_combined.reset_index(drop=True,inplace=True)
df_combined

,지역명,지역 코드,년월일,최저기온(℃),최고기온(℃),일강수량(mm),최대순간풍속(m/s)
0,서울,108,2020-01-02,-0.7,3.8,0.0,6.7
1,서울,108,2020-01-03,-3.4,4.6,0.0,5.8
2,서울,108,2020-01-04,-2.8,6.1,0.0,8.5
3,서울,108,2020-01-05,-3.2,6.6,0.0,4.4
4,서울,108,2020-01-06,0.6,4.6,1.8,8.8
...,...,...,...,...,...,...,...
11821,현충원,889,2024-03-16,1.7,19.8,0.0,7.2
11822,현충원,889,2024-03-17,4.2,12.9,0.0,10.0
11823,현충원,889,2024-03-18,-1.1,14.4,0.0,6.4
11824,현충원,889,2024-03-19,4.4,11.6,0.0,11.5


In [18]:
df_combined.to_csv('기상청 크롤링_2020~2024_2~3월 기온_서울.csv', index=True)